In [1]:
import pandas as pd
import pandas_datareader.data as web
import datetime
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.var_model import VARProcess
import statsmodels.api as sm
import numpy as np
from preproccessing import loadStooqData, loadYahooData
import matplotlib.pyplot as plt
import seaborn as sns



In [22]:
# long rate, short rate, cpi m, real gdp, stock index (SPASTT01USM661N), unempoyment
#usdpln, investment growth, consumption growth, unemployment, working population growth,  tradevalue
tickers = ['IRLTLT01USQ156N','IR3TIB01USQ156N','CPALTT01USQ657N','USAGDPRQPSMEI', 'LRUN64TTUSQ156S']

In [11]:
tickers = ['IR3TIB01PLM156N','IR3TIB01USM156N','CPALTT01USM657N','CPALTT01PLM657N']

In [12]:
start = datetime.datetime(2003, 1, 1)

end = datetime.datetime(2023, 12, 31)

source_data = web.DataReader(tickers, 'fred', start,end)


In [13]:
source_data.describe()

,IR3TIB01PLM156N,IR3TIB01USM156N,CPALTT01USM657N,CPALTT01PLM657N
count,250.000000,249.000000,249.000000,249.000000
mean,3.704399,1.681446,0.214471,0.268273
std,2.001726,1.759111,0.401120,0.495623
min,0.210000,0.090000,-1.915290,-0.500000
25%,1.720000,0.240000,-0.025438,0.000000
50%,4.150000,1.050000,0.211082,0.200000
75%,5.137500,2.590000,0.476323,0.500000
max,7.510000,5.490000,1.373608,3.300000


In [14]:
source_data.corr()

,IR3TIB01PLM156N,IR3TIB01USM156N,CPALTT01USM657N,CPALTT01PLM657N
IR3TIB01PLM156N,1.000000,0.459452,0.030939,0.201108
IR3TIB01USM156N,0.459452,1.000000,0.018848,0.054225
CPALTT01USM657N,0.030939,0.018848,1.000000,0.426388
CPALTT01PLM657N,0.201108,0.054225,0.426388,1.000000


In [31]:
data = pd.DataFrame({})
data = source_data.diff().dropna()

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 108 entries, 1993-01-01 to 2019-10-01
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   IRLTLT01USQ156N  108 non-null    float64
 1   IR3TIB01USQ156N  108 non-null    float64
 2   CPALTT01USQ657N  108 non-null    float64
 3   USAGDPRQPSMEI    108 non-null    float64
 4   LRUN64TTUSQ156S  108 non-null    float64
dtypes: float64(5)
memory usage: 5.1 KB


In [33]:
data.describe() 

,IRLTLT01USQ156N,IR3TIB01USQ156N,CPALTT01USQ657N,USAGDPRQPSMEI,LRUN64TTUSQ156S
count,108.000000,108.000000,108.000000,108.000000,108.000000
mean,-0.045833,-0.015154,-0.004599,-0.016759,-0.036467
std,0.363472,0.426360,0.786427,0.763854,0.286687
min,-0.863333,-1.793333,-3.991585,-2.782542,-0.554328
25%,-0.297500,-0.090833,-0.391209,-0.506764,-0.192249
50%,-0.026667,0.003333,-0.001563,-0.062177,-0.093655
75%,0.200833,0.152500,0.421124,0.368339,0.026220
max,1.016667,1.006667,2.330896,3.219333,1.411244


In [37]:
for ticker in tickers:
    print(ticker+ ': ', np.round(sm.tsa.stattools.adfuller(data[ticker])[1],4))

IRLTLT01USQ156N:  0.0
IR3TIB01USQ156N:  0.0
CPALTT01USQ657N:  0.0
USAGDPRQPSMEI:  0.0174
LRUN64TTUSQ156S:  0.0264


In [38]:
model = VAR(data)

c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)


In [39]:
result = model.fit(maxlags=1)
result.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 24, Aug, 2023
Time:                     22:57:31
--------------------------------------------------------------------
No. of Equations:         5.00000    BIC:                   -8.26855
Nobs:                     107.000    HQIC:                  -8.71415
Log likelihood:          -246.672    FPE:                0.000121286
AIC:                     -9.01794    Det(Omega_mle):     9.23292e-05
--------------------------------------------------------------------
Results for equation IRLTLT01USQ156N
                        coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------------
const                     -0.027420         0.034079           -0.805           0.421
L1.IRLTLT01USQ156N         0.260983         0.105856            2.465           0.014
L1.IR3TIB01USQ156N        -0.1

In [40]:
result.is_stable()

True

In [41]:
for i in range(5):
    print(result.test_causality(caused=i, causing=[0,1,2,3,4]))

<statsmodels.tsa.vector_ar.hypothesis_test_results.CausalityTestResults object. H_0: ['IRLTLT01USQ156N', 'IR3TIB01USQ156N', 'CPALTT01USQ657N', 'USAGDPRQPSMEI', 'LRUN64TTUSQ156S'] do not Granger-cause IRLTLT01USQ156N: reject at 5% significance level. Test statistic: 3.308, critical value: 2.232>, p-value: 0.006>
<statsmodels.tsa.vector_ar.hypothesis_test_results.CausalityTestResults object. H_0: ['IRLTLT01USQ156N', 'IR3TIB01USQ156N', 'CPALTT01USQ657N', 'USAGDPRQPSMEI', 'LRUN64TTUSQ156S'] do not Granger-cause IR3TIB01USQ156N: reject at 5% significance level. Test statistic: 17.580, critical value: 2.232>, p-value: 0.000>
<statsmodels.tsa.vector_ar.hypothesis_test_results.CausalityTestResults object. H_0: ['IRLTLT01USQ156N', 'IR3TIB01USQ156N', 'CPALTT01USQ657N', 'USAGDPRQPSMEI', 'LRUN64TTUSQ156S'] do not Granger-cause CPALTT01USQ657N: fail to reject at 5% significance level. Test statistic: 1.516, critical value: 2.232>, p-value: 0.183>
<statsmodels.tsa.vector_ar.hypothesis_test_results.C

In [42]:
result.fevd().summary()

FEVD for IRLTLT01USQ156N
     IRLTLT01USQ156N  IR3TIB01USQ156N  CPALTT01USQ657N  USAGDPRQPSMEI  LRUN64TTUSQ156S
0           1.000000         0.000000         0.000000       0.000000         0.000000
1           0.943145         0.017683         0.002497       0.035429         0.001246
2           0.923419         0.031138         0.004273       0.038774         0.002396
3           0.916710         0.035491         0.005603       0.038606         0.003590
4           0.913467         0.036290         0.006099       0.039797         0.004347
5           0.911974         0.036338         0.006220       0.040771         0.004696
6           0.911412         0.036316         0.006241       0.041207         0.004824
7           0.911238         0.036308         0.006243       0.041348         0.004863
8           0.911192         0.036306         0.006243       0.041384         0.004874
9           0.911182         0.036306         0.006243       0.041392         0.004876

FEVD for IR3TIB01

In [25]:
sim = result.simulate_var(steps=None,offset=None,nsimulations=100)

In [12]:
spy_source = loadStooqData('^SPX','1992-07-1')

In [13]:
spy = spy_source
spy.index = pd.to_datetime(spy.index)
spy = spy.resample('QS').last()
spy = spy.loc['1992-10-01':'2023-04-02']
spy = np.log(spy/spy.shift()).dropna()
#spy = sm.add_constant(spy, prepend=False)
#exog = sm.add_constant(source_data, prepend=False)

In [14]:
bond_source = pd.read_csv('VBMFX.csv')

In [15]:
bond = bond_source
bond = bond.set_index(bond['Date'])
bond = bond['Adj Close']
bond.index = pd.to_datetime(bond.index)
bond = bond.resample('QS').last()
bond = bond.loc['1992-10-01':'2023-04-02']
bond = np.log(bond/bond.shift(1)).dropna()


In [16]:
assets = pd.concat([spy,bond],axis=1)
assets.columns = ['SPY', 'BND']
#assets = assets.diff().dropna()

In [17]:
#np.round(sm.tsa.stattools.adfuller(assets['BND'])[1],4)
sm.tsa.stattools.adfuller(assets['SPY'])

(-10.562787407711081,
 7.65154907320737e-19,
 0,
 121,
 {'1%': -3.485585145896754,
  '5%': -2.885738566292665,
  '10%': -2.5796759080663887},
 -220.56991768257512)

In [18]:
model_a = VAR(assets,exog=data)

c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)


In [19]:
result = model_a.fit(2)

In [20]:
result.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 24, Aug, 2023
Time:                     20:36:56
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -12.8087
Nobs:                     120.000    HQIC:                  -13.0846
Log likelihood:           475.853    FPE:                1.72112e-06
AIC:                     -13.2733    Det(Omega_mle):     1.46651e-06
--------------------------------------------------------------------
Results for equation SPY
                     coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------------
const                  -0.000419         0.009632           -0.043           0.965
IRLTLT01USQ156N         0.066201         0.026863            2.464           0.014
IR3TIB01USQ156N         0.035644         0.021916     

In [21]:
result.is_stable()

True